### Step-by-Step Guide: Implementing Transfer Learning with EfficientNet
Ok, let’s now proceed with the actual implementation. We’ll go over the whole process step by step! Note that we provide really detailed comment for each code line we provide.

### Step 1: Import Necessary Libraries

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

Above we also imported to_categorical function that would let us convert the targets (classes that we try to classify) to one-hot-encoded vectors that we can use to fine-tune our CNN.

### Step 2: Load and Preprocess the Data

In [21]:
# Load Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

In [22]:
print(train_images.shape)

(60000, 28, 28)


We take just the first 1k images of this dataset 

In [23]:
train_images = train_images[:1000]
train_labels = train_labels[:1000]

So, the question is: how can we move from 1k 28 by 28 grayscale images to 1k 224 by 224 RGB images? Well, in TensorFlow & Keras it isn’t that hard! All we need to do add the third channel to all of the images and reshape them! So, if we do:

In [24]:
train_images = tf.expand_dims(train_images, axis=-1) #expand_dims() function we added the 3rd channel needed for EfficientNet-B0,
test_images = tf.expand_dims(test_images, axis=-1)

In [25]:
print(train_images.shape)

(1000, 28, 28, 1)


Reshape the images again (224 x 224)

In [17]:
train_images = tf.image.grayscale_to_rgb(train_images)
train_images = tf.image.resize(train_images, (224, 224))

test_images = tf.image.grayscale_to_rgb(test_images)
test_images = tf.image.resize(test_images, (224, 224))

In [18]:
print(train_images.shape)

(1000, 224, 224, 3)


Normalize our images to the range [0, 1]:

In [19]:
train_images = train_images / 255.0
test_images = test_images / 255.0

Convert labels to one-hot encoded ones

In [26]:
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

Next sum everything up:

In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Load Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Getting only 1k from the training portion of data
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Reshape the images to include the channel dimension (since EfficientNet expects 3D input)
train_images = tf.expand_dims(train_images, axis=-1)
test_images = tf.expand_dims(test_images, axis=-1)

# Resize images to match EfficientNet input size (224x224) and convert to 3 channels
train_images = tf.image.grayscale_to_rgb(train_images)
train_images = tf.image.resize(train_images, (224, 224))

test_images = tf.image.grayscale_to_rgb(test_images)
test_images = tf.image.resize(test_images, (224, 224))

# Normalize images to the range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

### Step 3: Import the Pre-Trained EfficientNetB0 Model

In [29]:
# Load the EfficientNetB0 model, excluding the top classification layers
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                            input_shape=(224, 224, 3))

# Freeze the base model to retain its pre-trained weights
base_model.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 12s 1us/step


### Step 4: Building the Custom Top Layer

The EfficientNet model will act as a feature extractor, meaning that it will extract information from the images but do not do the actual classification decision

In [30]:
# Create a custom top layer
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Converts the feature maps into a single vector
    layers.Dense(128, activation='relu'),  # Add a fully connected layer with 128 neurons
    layers.Dense(10, activation='softmax')  # Final layer with 10 outputs for 10 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Step 5: Training the Model

In [31]:
# Train the model
history = model.fit(train_images, train_labels, epochs=5,
                    validation_data=(test_images[:1000],
                    test_labels[:1000]))

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 26s 702ms/step - accuracy: 0.1043 - loss: 2.3538 - val_accuracy: 0.0950 - val_loss: 2.3265
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 665ms/step - accuracy: 0.0983 - loss: 2.3196 - val_accuracy: 0.1070 - val_loss: 2.3144
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 669ms/step - accuracy: 0.1045 - loss: 2.3099 - val_accuracy: 0.0950 - val_loss: 2.3089
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 679ms/step - accuracy: 0.1043 - loss: 2.3052 - val_accuracy: 0.0950 - val_loss: 2.3065
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 22s 706ms/step - accuracy: 0.1160 - loss: 2.3074 - val_accuracy: 0.0950 - val_loss: 2.3117


### Step 6: Fine-Tuning the Model (Optional)

In [33]:
# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images[:1000], test_labels[:1000]))

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 101s 3s/step - accuracy: 0.2173 - loss: 2.2461 - val_accuracy: 0.1110 - val_loss: 2.3187
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.2364 - loss: 1.9961 - val_accuracy: 0.1110 - val_loss: 2.3156
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.2759 - loss: 1.7907 - val_accuracy: 0.1110 - val_loss: 2.3159
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.3913 - loss: 1.5944 - val_accuracy: 0.1110 - val_loss: 2.3264
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.4756 - loss: 1.4894 - val_accuracy: 0.0660 - val_loss: 2.3305


Perform the model evaluation without fine-tuning that we’ve done above

### Step 7: Evaluating the model

In [34]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images[:1000],
        test_labels[:1000])

print(f"Test accuracy: {test_accuracy * 100:.2f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 320ms/step - accuracy: 0.0692 - loss: 2.3225
Test accuracy: 6.60
